### Buy And Hold
- 매수 후 일정 기간 동안 유지하고 매도

In [1]:
import pandas as pd

In [2]:
# 데이터 로드 
df = pd.read_csv("../../csv/AAPL.csv")

In [ ]:
df.head()

In [ ]:
df.set_index('Date')

In [5]:
df2 = pd.read_csv('../../csv/AAPL.csv', index_col=0)

In [6]:
df2.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.513393,0.515625,0.513393,0.513393,0.410525,117258400.0
1980-12-15,0.488839,0.488839,0.486607,0.486607,0.389106,43971200.0
1980-12-16,0.453125,0.453125,0.450893,0.450893,0.360548,26432000.0
1980-12-17,0.462054,0.464286,0.462054,0.462054,0.369472,21610400.0
1980-12-18,0.475446,0.477679,0.475446,0.475446,0.380182,18362400.0


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9715 entries, 1980-12-12 to 2019-06-24
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       9714 non-null   float64
 1   High       9714 non-null   float64
 2   Low        9714 non-null   float64
 3   Close      9714 non-null   float64
 4   Adj Close  9714 non-null   float64
 5   Volume     9713 non-null   float64
dtypes: float64(6)
memory usage: 531.3+ KB


In [8]:
# 결측치가 보인다? -> 결측치인 데이터를 확인 
# 결측치가 가장 많은 컬럼의 데이터를 기준으로 확인 
flag = df2['Volume'].isna()

df2.loc[flag, ]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1981-08-10,NaN,NaN,NaN,NaN,NaN,NaN
1981-08-21,0.363839,0.363839,0.359375,0.359375,0.287367,NaN


In [9]:
# 결측치를 제외한 데이터를 가지고 BuyandHold의 백테스팅
df = df2.loc[~flag,]

In [ ]:
df.info()

In [11]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,9713.000000,9713.000000,9713.000000,9713.000000,9713.000000,9.713000e+03
mean,28.217776,28.490616,27.932493,28.218617,26.394897,8.688370e+07
std,49.684194,50.120923,49.248888,49.697758,47.936067,8.646964e+07
min,0.198661,0.198661,0.196429,0.196429,0.157070,3.472000e+05
25%,1.058036,1.080000,1.036429,1.060268,0.914825,3.362830e+07
50%,1.686429,1.723214,1.651786,1.687500,1.446356,5.871600e+07
75%,27.857143,28.114286,27.402857,27.757143,24.316063,1.083768e+08
max,230.779999,233.470001,229.779999,232.070007,229.392090,1.855410e+09


In [12]:
import numpy as np

In [16]:
# 결측치나 양의 무한대, 음의 무한대가 포함되어있는 데이터를 확인 
flag = df.isin(
    [np.nan, np.inf, -np.inf]
).any(axis=1)

In [18]:
# 결측치나 양의 무한대 음의 무한대를 제거 
df = df.loc[~flag, ]

In [19]:
# 수정종가 데이터만 두고 모두 제거 -> 수정종가만 따로 추출해서 데이터프레임
price_df = df[ ['Adj Close'] ]

In [24]:
# 새로운 파생변수 일별 수익율
# 오늘의 수정종가 / 전 행의 수정종가 shift(1)
price_df['daily_rtn'] = (
    price_df['Adj Close'] / price_df['Adj Close'].shift()
).fillna(1)

C:\Users\moons\AppData\Local\Temp\ipykernel_20288\1627393438.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_df['daily_rtn'] = (


In [25]:
# 수익율 계산 
# 투자 매수 일 -> 다음날 부터 일별 수익율을 누적곱을 해서 
# 매도일까지 결과를 누적수익율이라 계산
rtn = 1

for idx in range(1, len(price_df)):
    rtn *= price_df.iloc[idx, -1]

In [26]:
rtn

485.1592424334637